Python 3 scripts for creating Gaussian input files to calculate counterpoise-corrected 2 and 3 body interactions energies. Designed for a system of a protein active site with any number of residues and a ligand bound to it non-covalently. Gaussian 16 does not support solvents in counterpoise calculations, so these scripts allow the calculation of counterpoise energies with implicit solvent.

Input is the total number of atoms in the file (in reality this number can be the total number of atoms or any number greater than that), the number of fragments including the ligand, and a filename for a file from which to read, in order:
    1. a single line containing a space separated list of the number of atoms in each fragment,
    2. a single line containing a space separated list of the charges of each fragment
    3. the xyz coordinates of all atoms in fragment order (same order as the lists in 1 and 2). For example, all of the            atoms in fragment 1, then all of the atoms in fragment 2, etc.
The first script defines an array for the multiplicities of each fragment. In most cases this will be 1, but if you have other multiplicies, you can modify the script to use the multiplicities array.
***A sample input file is provided, called 3-body-sample***
  
This works for non-peptide-bonded fragments. If your active site has peptide-bonded fragments, you will have to separate them manually, cap the ends, and create input files for them with the other residues in the system.

User modifications/customizations on first script:

Gaussian comment line - line 11: modify snippet of input filename to be used
Output filename - modify line 15
Methods, basis set, solvents, etc - modify lines 28-32

The second script is nearly identical to the first but includes the ligand and the first amino acid (fragment 2) in every triple.

The class molprint is set up for 3 body interactions, but can be expanded to any number of fragments. 

In [ ]:
#===============================================================
#3 body general, with first molecule (ligand) in every triple
#===============================================================
import csv
import numpy

totn, totf = [int(x) for x in input("enter total number of atoms and number of fragments: ").split()] 
filename = input("enter filename: ")

#first 3 letters of input file to use as a comment =====
filetemp = filename[0:3]
#======================================================

#modify here for output file name =====================
outfile = "3-Body.txt"
#======================================================
print(outfile)
geom = numpy.empty((totn,4)).tolist()
cha = numpy.empty((totf)).tolist()
mult = numpy.empty((totf)).tolist()
molsize = numpy.empty((totf)).tolist()

class molprint():
    def printheader():
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")

            outwriter.writerow(["%chk=3b.chk"])
            outwriter.writerow(["%mem=16GB"])
            outwriter.writerow(["%nprocshared=16"])
            outwriter.writerow(["#", "aug-cc-pVDZ", "CAM-B3LYP", "EmpiricalDispersion=GD3BJ",
                               "SCRF(Solvent=water)"])
            outwriter.writerow([])
            outwriter.writerow([filetemp,"interactions"])
            outwriter.writerow([])
    
    #=========================================================
    #l1 and l2 are mol 1 limits, l3 and l4 are mol2 limts, etc
    #=========================================================    
    def discontinuousBq(l1,l2,l3,l4,l5,l6,charge,multl,notbq):
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")
            outwriter.writerow([charge,multl])
            for i in range(l1,l2,1):
                if notbq == 1:
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:                
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l3,l4,1):
                if notbq == 2:        
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l5,l6,1):
                if notbq == 3:        
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            outwriter.writerow([])
            outwriter.writerow(["--Link1--"])
    
    def discontinuous(l1,l2,l3,l4,l5,l6,charge):
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")
            outwriter.writerow([charge,1])
            for i in range(l1,l2,1):
                outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l3,l4,1):        
                outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l5,l6,1):       
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            outwriter.writerow([])
            outwriter.writerow(["--Link1--"])
            
#=================================================================================================
#read input
#=================================================================================================
with open(filename) as MolecSpec:
    molreader = csv.reader(MolecSpec, skipinitialspace=True, delimiter=' ' )
    line = 0
    
    for row in molreader:
        if line == 0: 
            for i in range(totf):
                molsize[i] = int(row[i])
            line += 1
        elif line == 1:
            for i in range(totf):
                cha[i] = int(row[i])
            line += 1
        else:
            for j in range(0,4,1):
                #print(line,j)
                geom[line-2][j] = row[j]
            line += 1

mult = [1]*totf
print(molsize,cha, mult)

#=================================================================================================
#input complete; now define an instance of molprint and call the fragments
#for molecule n the start is sum(molsize[0:n-1:1])
#=================================================================================================
threebodyprint = molprint

i = 0
for j in range(1,totf,1):
    for k in range(j+1,totf,1):
        
        #print total
        threebodyprint.printheader()
        threebodyprint.discontinuous(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[i]+cha[j]+cha[k])
        
        #print mol1
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[i],mult[i],1)
        
        #print mol2
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[j],mult[j],2)
        
        #print mol3
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[k],mult[k],3)  

In [ ]:
#=========================================================================================
#3 body, with first molecule (ligand) and second molecule (1st amino acid) in every triple
#=========================================================================================
import csv
import numpy

totn, totf = [int(x) for x in input("enter total number of atoms and number of fragments: ").split()] 
filename = input("enter filename: ")
filetemp = filename[0:3]

outfile = "3body.txt"
print(outfile)
geom = numpy.empty((totn,4)).tolist()
cha = numpy.empty((totf)).tolist()
molsize = numpy.empty((totf)).tolist()

class molprint():
    def printheader():
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")

            outwriter.writerow(["%chk=3b.chk"])
            outwriter.writerow(["%mem=16GB"])
            outwriter.writerow(["%nprocshared=16"])
            outwriter.writerow(["#", "aug-cc-pVDZ", "CAM-B3LYP", "EmpiricalDispersion=GD3BJ",
                               "SCRF(Solvent=water)"])
            outwriter.writerow([])
            outwriter.writerow([filetemp,"chain"])
            outwriter.writerow([])
    
    #=========================================================
    #l1 and l2 are mol 1 limits, l3 and l4 are mol2 limts, etc
    #=========================================================    
    def discontinuousBq(l1,l2,l3,l4,l5,l6,charge,notbq):
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")
            outwriter.writerow([charge,1])
            for i in range(l1,l2,1):
                if notbq == 1:
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:                
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l3,l4,1):
                if notbq == 2:        
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l5,l6,1):
                if notbq == 3:        
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
                else:
                    tempstring = str(geom[i][0])+"-Bq"
                    outwriter.writerow([tempstring,geom[i][1],geom[i][2],geom[i][3]])
            outwriter.writerow([])
            outwriter.writerow(["--Link1--"])
    
    def discontinuous(l1,l2,l3,l4,l5,l6,charge):
        with open(outfile,mode="a", newline="") as outputfile:
            outwriter = csv.writer(outputfile, delimiter=" ")
            outwriter.writerow([charge,1])
            for i in range(l1,l2,1):
                outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l3,l4,1):        
                outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            for i in range(l5,l6,1):       
                    outwriter.writerow([geom[i][0],geom[i][1],geom[i][2],geom[i][3]])
            outwriter.writerow([])
            outwriter.writerow(["--Link1--"])
            
#=================================================================================================
#read input
#=================================================================================================
with open(filename) as MolecSpec:
    molreader = csv.reader(MolecSpec, skipinitialspace=True, delimiter=' ' )
    line = 0
    
    for row in molreader:
        if line == 0: 
            for i in range(totf):
                molsize[i] = int(row[i])
            line += 1
        elif line == 1:
            for i in range(totf):
                cha[i] = int(row[i])
            line += 1
        else:
            for j in range(0,4,1):
                #print(line,j)
                geom[line-2][j] = row[j]
            line += 1

print(molsize,cha)

#=================================================================================================
#input complete; now define an instance of molprint and call the fragments
#for molecule n the start is sum(molsize[0:n-1:1])
#=================================================================================================
threebodyprint = molprint

i = 0
for j in range(1,2,1):
    for k in range(j+1,totf,1):
        
        #print total
        threebodyprint.printheader()
        threebodyprint.discontinuous(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[i]+cha[j]+cha[k])
        
        #print mol1
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[i],1)
        
        #print mol2
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[j],2)
        
        #print mol3
        threebodyprint.printheader()
        threebodyprint.discontinuousBq(sum(molsize[0:i:1]),sum(molsize[0:i+1:1]),sum(molsize[0:j:1]),
                                     sum(molsize[0:j+1:1]),sum(molsize[0:k:1]),sum(molsize[0:k+1:1]),
                                     cha[k],3)  